In [2]:
import time
import os
import sys
import csv
import math
import json
import shutil
import random
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.profiler
import torchvision.datasets as datasets
from torch.utils.data import DataLoader as dataloader
import torchvision.models as models

sys.path.insert(1, 'utils')
from HelperFunctions import *
from pytorch_resnet import *
from randaugment import RandAugmentMC

import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

cpu_device = torch.device("cpu:0")

from efficientnet_lite import build_efficientnet_lite

In [7]:
colour = 'Green'
config = dict(
        epochs=20,
        classes=['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil'],
        image_size=160,
        batch_size=64,
        valid_batch_size = 12,
        batch_size_latency=1,
        save = 'Models/baseline/temp/'+ colour,
        save_suffix = '_profiling_'+colour,
        lr = 1.5e-4,
        patience=15,
        delta=0.001,
        lr_lambda = 0.9,
        transforms = dict(n=15,m=10,norm=None),
        architecture_list = [#'resnet18',
                            # 'resnet34',
                            # 'resnet50', 
                            # 'regnet_y_400mf',
                            #'mobilenet_v3_small'
                            'mobilenet_v2',
                            # 'shufflenet_v2_x0_5',
                            # 'regnet_x_400mf',
                            # 'regnet_x_800mf',
                            # 'regnet_y_400mf',
                            # 'regnet_y_800mf'
                            ],
        data_path = 'T:/Haematology FYP/Data/WBC_Classification_3172_c/',
        tags = [colour,'Training','v3'],
        profiling = False,
        num_workers=6
    )

In [8]:
#model = CustomRegNet()
#fusion_regnet(model)
model = create_model('mobilenet_v2', cpu_device)
check_point = torch.load('Models/baseline/temp/Greenmobilenet_v2_profiling_Green.pt')
model.load_state_dict(check_point['model_state_dict'])
model.to(cpu_device)

_, _, loader = datamaker('Green', config)
img, true = next(iter(loader))
model.eval()
output = model(img)
print(calculate_accuracy(output,true))

Train samples: 4563    Batch size: 71
Validation samples: 782     Batch size: 65
Testing samples: 833     Batch size: 69
tensor(0.1667)


In [10]:
img = next(iter(loader))[0][0]
print(img.shape)

torch.Size([3, 160, 160])


In [3]:
model = build_efficientnet_lite('efficientnet_lite0', 5)

In [13]:
for n,m in model.named_children():
    if n not in ['fc','linear','classifier']:
        for param in m.parameters():
            param.requires_grad = False


In [6]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[-1.5363e-01,  5.8211e-02,  4.2194e-02],
          [ 1.1636e-01, -1.0119e-01, -1.0991e-01],
          [ 3.8095e-02,  4.0631e-02,  5.6153e-02]],

         [[-1.0124e-01,  2.4318e-02,  2.6210e-03],
          [ 4.5932e-03,  8.3366e-02,  6.0831e-03],
          [ 2.1773e-02,  1.0586e-01, -1.0541e-02]],

         [[-1.1606e-01, -8.9147e-02,  3.4734e-02],
          [-1.1547e-01,  7.8642e-02, -1.5857e-01],
          [-3.8042e-02, -1.1448e-01,  1.1739e-01]]],


        [[[ 1.0965e-01,  8.1760e-02,  5.6958e-03],
          [ 3.1830e-02,  1.1793e-01, -3.3715e-02],
          [ 6.0364e-03, -8.9317e-02,  4.6962e-02]],

         [[-1.3589e-01,  1.4838e-01, -1.5263e-01],
          [ 1.0249e-02,  9.9642e-02,  8.8295e-02],
          [-9.2252e-02, -1.1964e-01, -1.3273e-01]],

         [[-1.3883e-01,  3.8407e-02, -1.4824e-02],
          [-7.4948e-02,  5.4851e-02,  1.4117e-02],
          [ 6.8196e-02,  9.4815e-02, -2.3262e-01]]],


        [[[ 8.0134e-02,  3.7116e-02,  2.7239